# Pytorch tutorial


## What is pytorch?
- Numpy를 대체하며 GPU연산을 지원하는 tensor 구조 지원  
CUDA(NDVIA에서 gpu를 통한 연산을 위해 만든 API)와 cuDNN(CUDA를 활용하여 딥러닝 gpu연산을 지원하는 API)을 활용하여 연산 가속  
일반적으로 CUDA와 cuDNN을 활용하는 연산은 CPU연산의 15배정도 가속을 가져옴. 


- backward() 함수를 통한 그래프 미분 연산 지원

## Pytorch vs Tensorflow

#### 둘 모두 GPU를 활용하는 framework
- Tensorflow  
`Define and Run`방식  
연산 그래프를 미리 만들어두고, 실제 연산시 값을 전달하여 결과를 얻음(정적)  
직관적이지 않고 그래프를 정의하는 부분과 실행하는 부분이 분리되어 코드가 길어짐  
최적화에서 장점


- Pytorch  
`Define by Run` 방식  
연산 그래프를 미리 만들어두지 않고 값이 할당되어 전달되는 과정에서 그래프가 작성(동적)  
직관적이고 간단한 코드  
(Pytorch측의 주장으로는)Tensorflow보다 평균 2.5배정도 빠른 속도(적어도 밀리지는 않음)

## Tensor

Tensor: Numpy의 ndarray와 유사한 matrix자료구조. GPU연산 가속이 가능

In [1]:
import torch
import numpy as np

### Tensor 생성

In [2]:
not_initialized = torch.empty(3, 4)
not_initialized

tensor([[-2.5214e-11,  3.0918e-41,  3.3631e-44,  0.0000e+00],
        [        nan,  3.0918e-41,  1.1578e+27,  1.1362e+30],
        [ 7.1547e+22,  4.5828e+30,  1.2121e+04,  7.1846e+22]])

In [3]:
random_initialized = torch.rand(3, 4)
random_initialized

tensor([[0.4638, 0.9163, 0.7069, 0.0338],
        [0.4469, 0.4418, 0.6278, 0.8632],
        [0.4040, 0.6181, 0.8287, 0.5062]])

In [4]:
zero_initialized = torch.zeros(3, 4)
zero_initialized

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [5]:
list_initialized = torch.tensor([[1, 2],[3, 4]])
list_initialized

tensor([[1, 2],
        [3, 4]])

In [6]:
arange_lnitiaized = torch.arange(10)
arange_lnitiaized

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

### Tensor와 numpy간의 변환

In [7]:
x = torch.ones(10)
x, type(x)

(tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), torch.Tensor)

In [8]:
y = x.numpy()
y, type(y)

(array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32), numpy.ndarray)

In [9]:
z = torch.from_numpy(y)
z

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

#### 주의사항 - tensor와 ndarray는 메모리공간을 공유함

In [10]:
x.add_(1)

tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

In [11]:
x

tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

In [12]:
y, z

(array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.], dtype=float32),
 tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]))

### Tensor 특성

In [13]:
list_initialized.shape, list_initialized.ndim

(torch.Size([2, 2]), 2)

In [14]:

list_initialized.size()

torch.Size([2, 2])

### Tensor reshape vs view

-  두 함수는 기본적으로 같은 기능을 하나, view는 shallow copy이다.
-  그러나, reshape이 deep copy인 것은 아니다. reshape은 상황에 따라 deep/shallow를 넘나든다.
-  Reshape은 Contiguous한 operate를 지원하나, view는 지원하지 않는다.

In [15]:
list_initialized.t()

tensor([[1, 3],
        [2, 4]])

In [16]:
list_initialized.t().reshape(-1, 1)

tensor([[1],
        [3],
        [2],
        [4]])

In [18]:
list_initialized.t().view(-1, 1) # Error!

RuntimeError: ignored

### Tensor의 배열 연산

인덱싱, 슬라이싱, 선형대수 등 각종 연산 numpy와 동일하게 수행 가능

#### 기본연산

In [19]:
x = torch.arange(1, 11).reshape(2, 5)
y = torch.arange(1, 20, step=2).reshape(2, 5)
x, y

(tensor([[ 1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10]]), tensor([[ 1,  3,  5,  7,  9],
         [11, 13, 15, 17, 19]]))

In [20]:
x + y, torch.add(x, y)

(tensor([[ 2,  5,  8, 11, 14],
         [17, 20, 23, 26, 29]]), tensor([[ 2,  5,  8, 11, 14],
         [17, 20, 23, 26, 29]]))

In [21]:
x - y, torch.sub(x, y)

(tensor([[ 0, -1, -2, -3, -4],
         [-5, -6, -7, -8, -9]]), tensor([[ 0, -1, -2, -3, -4],
         [-5, -6, -7, -8, -9]]))

In [22]:
x * y

tensor([[  1,   6,  15,  28,  45],
        [ 66,  91, 120, 153, 190]])

In [23]:
y % x

tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])

#### 행렬곱

In [24]:
x.matmul(y.reshape(5, 2))

tensor([[175, 205],
        [400, 480]])

#### 바꿔치기(in-place) 와 반환하기(out-of-place)

In [25]:
x

tensor([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10]])

In [26]:
x.add(3)

tensor([[ 4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13]])

In [27]:
x

tensor([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10]])

In [28]:
x.add_(3)

tensor([[ 4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13]])

In [29]:
x

tensor([[ 4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13]])

#### 헷갈리는 경우

In [30]:
z = x.numpy()
x = x.add(3)
print('x:',x)
print('z:',z)
print('========================')
x.add_(3)
print('x:', x)
print('z:', z)

x: tensor([[ 7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16]])
z: [[ 4  5  6  7  8]
 [ 9 10 11 12 13]]
x: tensor([[10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]])
z: [[ 4  5  6  7  8]
 [ 9 10 11 12 13]]


#### Indexing and slicing

In [31]:
x[0, 3]

tensor(13)

In [32]:
x[:, 1:4]

tensor([[11, 12, 13],
        [16, 17, 18]])

### Aggregation

In [33]:
x

tensor([[10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]])

In [34]:
x.sum()

tensor(145)

In [35]:
x.sum(dim=0), x.sum(axis=0)

(tensor([25, 27, 29, 31, 33]), tensor([25, 27, 29, 31, 33]))

In [36]:
x.sum(dim=1), x.sum(axis=1)

(tensor([60, 85]), tensor([60, 85]))

#### Can't use mean() on integers

In [37]:
x.dtype, x

(torch.int64, tensor([[10, 11, 12, 13, 14],
         [15, 16, 17, 18, 19]]))

In [38]:
try:
    print(x.mean(dim=0))
except Exception as exc:
    print(exc)

Can only calculate the mean of floating types. Got Long instead.


In [39]:
# x = torch.arange(1, 11, dtype=float).reshape(2, 5)
x = x.float()
x

tensor([[10., 11., 12., 13., 14.],
        [15., 16., 17., 18., 19.]])

In [40]:
x.mean(), x.mean(dim=0)

(tensor(14.5000), tensor([12.5000, 13.5000, 14.5000, 15.5000, 16.5000]))

### Broadcasting

In [41]:
x = torch.arange(1, 11).reshape(1, -1)
y = torch.arange(1, 20, step=2).reshape(-1, 1)

In [42]:
x + y

tensor([[ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
        [ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
        [ 6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
        [ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17],
        [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        [12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
        [14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
        [16, 17, 18, 19, 20, 21, 22, 23, 24, 25],
        [18, 19, 20, 21, 22, 23, 24, 25, 26, 27],
        [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]])

## AUTOGRAD: 자동미분

`pytorch` 신경망의 중심. `backward()`를 가능케 하는 패키지  
`Tensor`의 모든 연산에 대해 자동 미분을 제공

`pytorch.Tensor` class에는 `.required_grad` 속성이 존재  
`.required_grad`를 True로 세팅하면 해당 tensor를 기반으로 이루어신 모든 연산을 추적(track)하기 시작함  
모든 연산이 완료된 후, `backward()`를 호출하면 모든 gradient가 자동으로 계산(`Tensor.grad` 속성에 gradient가 누적됨)

Autograd의 다른 중요 class는 `Function`class  
`Tensor`와 `Function`class는 연결되어 있으며, 모든 연산을 encode하여 acyclic graph를 생성  
각각의 tensor는 `.grad_fn`속성을 가지며 이는 `Tensor`를 생성한 `Function`을 참조함

In [43]:
x = torch.ones(5, 2, requires_grad=True)
# x = torch.arange(10, dtype=torch.float32).reshape(5, 2)
# x.requires_grad = True
x

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]], requires_grad=True)

In [44]:
y = x + 10

In [45]:
y

tensor([[11., 11.],
        [11., 11.],
        [11., 11.],
        [11., 11.],
        [11., 11.]], grad_fn=<AddBackward0>)

y는 연산의 결과로 생성된 tensor이므로 grad_fn을 가짐

In [46]:
y.grad_fn

In [47]:
z = y * y * 10

In [48]:
z

tensor([[1210., 1210.],
        [1210., 1210.],
        [1210., 1210.],
        [1210., 1210.],
        [1210., 1210.]], grad_fn=<MulBackward0>)

In [49]:
out = z.mean()

In [50]:
z, out

(tensor([[1210., 1210.],
         [1210., 1210.],
         [1210., 1210.],
         [1210., 1210.],
         [1210., 1210.]], grad_fn=<MulBackward0>),
 tensor(1210., grad_fn=<MeanBackward0>))

### 변화도(Gradient) 계산)

In [51]:
out.backward(retain_graph=True)

In [52]:
print(x.grad)

tensor([[22., 22.],
        [22., 22.],
        [22., 22.],
        [22., 22.],
        [22., 22.]])


#### Jacobian matrix 활용

In [53]:
x = torch.ones(5, 2, requires_grad=True)
y = x + 10
z = y * y * 10
out = z.mean(axis=1)
out

tensor([1210., 1210., 1210., 1210., 1210.], grad_fn=<MeanBackward1>)

In [54]:
x = torch.ones(5, 2, requires_grad=True)
y = x + 10
y = y * y * 10
y = y.mean(axis=1)
y

tensor([1210., 1210., 1210., 1210., 1210.], grad_fn=<MeanBackward1>)

In [55]:
y.backward(torch.tensor([1., 3.,1.,1.,1.]))

In [56]:
print(x.grad)

tensor([[110., 110.],
        [330., 330.],
        [110., 110.],
        [110., 110.],
        [110., 110.]])


## DataLoader

torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False,  num_workers=0,...)

데이터를 학습모델에 통째로 넣지 않고 batch 단위로 나누어 학습시킬 때 활용.  
모든 데이터를 나누고 나눈 데이터를 차례로 넣어주는 과정을 대신해줌
- 용어  
 - epoch: 모든 training data 혹은 모든 training data를 학습시킨 상태
 - batch: batch_size만큼의 training data만 학습시키기 위한 소분 데이터


In [57]:
x = [[73, 80, 75],
   [93, 88, 93],
   [89, 91, 90],
   [96, 98, 100],
   [73, 66, 70]]
x = torch.FloatTensor(x)
x

tensor([[ 73.,  80.,  75.],
        [ 93.,  88.,  93.],
        [ 89.,  91.,  90.],
        [ 96.,  98., 100.],
        [ 73.,  66.,  70.]])

In [58]:
data_loader = torch.utils.data.DataLoader(x,
                            batch_size=2, 
                            shuffle=True, 
                            num_workers=0)

In [59]:
epochs = 2
for epoch in range(epochs):
    for step, batch in enumerate(data_loader):
        print(step, batch)

0 tensor([[89., 91., 90.],
        [93., 88., 93.]])
1 tensor([[ 73.,  66.,  70.],
        [ 96.,  98., 100.]])
2 tensor([[73., 80., 75.]])
0 tensor([[93., 88., 93.],
        [73., 66., 70.]])
1 tensor([[ 96.,  98., 100.],
        [ 73.,  80.,  75.]])
2 tensor([[89., 91., 90.]])


## CUDA Tensor

### GPU로 이동

In [60]:
if torch.cuda.is_available():
    x = x.to(torch.device('cuda'))
    print(type(x))
    print(x.device)

<class 'torch.Tensor'>
cuda:0


#### 생성과 함께 gpu에 할당

In [61]:
if torch.cuda.is_available():
    k = torch.full((2, 3), 1, device=torch.device('cuda'))
    print(type(k))
    print(k.device)

<class 'torch.Tensor'>
cuda:0


### CPU로 이동

In [62]:
if torch.cuda.is_available():
    x = x.to(torch.device('cpu'))
    print(type(x))
    print(x.device)

<class 'torch.Tensor'>
cpu
